<a href="https://colab.research.google.com/github/keerthireddy-28/GenerativeAI/blob/main/2303A52263_week6_Ass6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. (1 ponto) Design a multilayer ANN architecture according to the requirements shown below.
Train, test, save (.h5) and deploy the model to predict the housing price using Keras deep
learning library
2. (1 ponto) Calculate training and testing error metrics
3. (1 ponto) Build the application by loading the saved ANN model.
Tabela 1: ANN Architecture
Layer Neurons Activation Function

Hidden Layer - 1 15 tanh

Hidden Layer - 2 20 tanh

Hidden Layer - 3 15 tanh

Tabela 2: Training Parameters
loss function epochs batch size error metric Optimizer
Mean Square Error 100 16 Mean Square Error SGD

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.preprocessing import OneHotEncoder

url = '/content/Housing (1).csv'
df = pd.read_csv(url)

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

categorical_features = X_train.select_dtypes(include=['object']).columns

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_features_train = encoder.fit_transform(X_train[categorical_features])
encoded_features_test = encoder.transform(X_test[categorical_features])

encoded_df_train = pd.DataFrame(encoded_features_train, columns=encoder.get_feature_names_out(categorical_features), index=X_train.index)
encoded_df_test = pd.DataFrame(encoded_features_test, columns=encoder.get_feature_names_out(categorical_features), index=X_test.index)

X_train = X_train.drop(categorical_features, axis=1)
X_test = X_test.drop(categorical_features, axis=1)
X_train = pd.concat([X_train, encoded_df_train], axis=1)
X_test = pd.concat([X_test, encoded_df_test], axis=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import MeanSquaredError

model = Sequential()

model.add(Dense(15, input_dim=X_train.shape[1], activation='tanh'))

model.add(Dense(20, activation='tanh'))

model.add(Dense(15, activation='tanh'))

model.add(Dense(1))

model.compile(loss='mean_squared_error',
              optimizer=SGD(),
              metrics=[MeanSquaredError()])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
# Train the model
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=16,
                    validation_data=(X_test, y_test))


Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 9803192598528.0000 - mean_squared_error: 9803192598528.0000 - val_loss: 5234817499136.0000 - val_mean_squared_error: 5234817499136.0000
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 2719831556096.0000 - mean_squared_error: 2719831556096.0000 - val_loss: 5284475437056.0000 - val_mean_squared_error: 5284475437056.0000
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3664485285888.0000 - mean_squared_error: 3664485285888.0000 - val_loss: 5283190407168.0000 - val_mean_squared_error: 5283190407168.0000
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3048083554304.0000 - mean_squared_error: 3048083554304.0000 - val_loss: 5385974972416.0000 - val_mean_squared_error: 5385974972416.0000
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3416314609664.0000 - mean_squared_error: 3416314609664.0000 - val_loss: 5998322384896.0000 - val_mean_squared_error: 5998322384896.0000
Epoch 6/100
28/28 ━━━━━━

In [27]:
# Calculate the training error
train_loss, train_mse = model.evaluate(X_train, y_train)
print(f"Training MSE: {train_mse}")

# Calculate the testing error
test_loss, test_mse = model.evaluate(X_test, y_test)
print(f"Testing MSE: {test_mse}")


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3135833899008.0000 - mean_squared_error: 3135833899008.0000 
Training MSE: 3083425021952.0
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 5309790158848.0000 - mean_squared_error: 5309790158848.0000
Testing MSE: 5117673734144.0


In [28]:
# Save the trained model
model.save('housing_price_model.h5')


In [31]:
from tensorflow.keras.models import load_model

loaded_model = load_model('housing_price_model.h5')

predictions = loaded_model.predict(X_test)

for actual, predicted in zip(y_test[:10], predictions[:10]):
    print(f"Actual: {actual}, Predicted: {predicted[0]}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Actual: 4060000, Predicted: 4835696.5
Actual: 6650000, Predicted: 4835696.5
Actual: 3710000, Predicted: 4835696.5
Actual: 6440000, Predicted: 4835696.5
Actual: 2800000, Predicted: 4835696.5
Actual: 4900000, Predicted: 3546975.0
Actual: 5250000, Predicted: 4455810.5
Actual: 4543000, Predicted: 4835696.5
Actual: 2450000, Predicted: 4835696.5
Actual: 3353000, Predicted: 4835696.5
